# FinBot: Financial Agent Tool-Calling Fine-Tuning

This notebook implements a **Financial Agent Fine-tuning Experiment** demonstrating how targeted fine-tuning can improve tool-calling accuracy for Small Language Models (SLMs).

## Research Context

Based on findings from ["Small Language Models for Efficient Agentic Tool Calling"](https://arxiv.org/abs/2512.15943), which showed that a fine-tuned 350M parameter model achieved **77.55% pass rate** on ToolBench—outperforming ChatGPT-CoT (26%) and ToolLLaMA (30.18%)—this experiment explores whether similar gains can be achieved for domain-specific financial tool calling.

## Methodology: Three-Tier Query Complexity

To properly evaluate fine-tuning effectiveness, we use **three categories of test queries**:

| Category | Description | Challenge Level |
|----------|-------------|-----------------|
| **Direct** | Single tool, explicit parameters | Easy - Base model likely succeeds |
| **Multi-Step** | Sequential tool chains, comparative analysis | Medium - Requires planning |
| **Domain-Specific** | Financial reasoning, implicit parameters | Hard - Requires domain knowledge |

## Experiment Flow

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│  1. Load Base   │ -> │  2. Generate    │ -> │  3. Train/Eval  │ -> │  4. SFT Train   │ -> │  5. Eval TUNED  │
│   Qwen2.5-3B    │    │  Synthetic Data │    │  Split Dataset  │    │  (Tool Calling) │    │  (Compare)      │
└─────────────────┘    └─────────────────┘    └─────────────────┘    └─────────────────┘    └─────────────────┘
                              │                       │
                              v                       v
                    ┌─────────────────────┐  ┌─────────────────────┐
                    │ Query Types:        │  │ Evaluation Metrics: │
                    │ • Direct (simple)   │  │ • Tool selection    │
                    │ • Multi-step        │  │ • Sequence accuracy │
                    │ • Domain-specific   │  │ • Parameter accuracy│
                    └─────────────────────┘  └─────────────────────┘
```

## Expanded Tool Set (10 Tools)

We expand from 4 to 10 tools to increase complexity and create tool selection ambiguity:

| Tool | Signature | yfinance Mapping | Category |
|------|-----------|------------------|----------|
| `get_current_price` | `(ticker: str)` | `fast_info['last_price']` | Price |
| `get_company_profile` | `(ticker: str)` | `info` | Fundamentals |
| `get_historical_data` | `(ticker: str, period: str)` | `history(period)` | Historical |
| `get_analyst_recommendations` | `(ticker: str)` | `recommendations` | Analysis |
| `get_earnings_history` | `(ticker: str)` | `earnings_history` | Earnings |
| `get_dividend_history` | `(ticker: str)` | `dividends` | Income |
| `get_insider_trades` | `(ticker: str)` | `insider_transactions` | Insider |
| `get_institutional_holders` | `(ticker: str)` | `institutional_holders` | Ownership |
| `get_news` | `(ticker: str)` | `news` | News |
| `compare_stocks` | `(tickers: List[str], metric: str)` | Multiple calls | Comparative |

## Evaluation Criteria

1. **Tool Selection Accuracy**: Did the model choose the correct tool(s)?
2. **Sequence Accuracy**: For multi-step queries, is the tool order correct?
3. **Parameter Accuracy**: Are ticker symbols, periods, and other parameters correct?
4. **Overall Pass Rate**: Complete correctness across all dimensions

## Section 1: Setup & Configuration

In [1]:
import os
import re
import json
import random
import warnings
from pathlib import Path
from dataclasses import dataclass
from typing import Literal, Optional, List, Dict, Any
from datetime import datetime
import time

import pandas as pd
from datasets import Dataset
from dotenv import load_dotenv
from tqdm import tqdm

warnings.filterwarnings('ignore')

# =============================================================================
# PROJECT PATHS
# =============================================================================
PROJECT_ROOT = Path().absolute().parent.parent
NOTEBOOKS_DIR = PROJECT_ROOT / "notebooks" / "tuning"
DATA_DIR = PROJECT_ROOT / "data"
MODELS_DIR = PROJECT_ROOT / "models"
MLRUNS_DIR = NOTEBOOKS_DIR / "mlruns"
OUTPUT_DIR = MODELS_DIR / "checkpoints" / "finbot-tool-calling"
SYNTHETIC_DATA_FILE = DATA_DIR / "processed" / "finbot_synthetic_data.json"

# Create directories
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
(DATA_DIR / "processed").mkdir(parents=True, exist_ok=True)
MLRUNS_DIR.mkdir(parents=True, exist_ok=True)

# Load environment
env_path = PROJECT_ROOT / ".env"
load_dotenv(env_path if env_path.exists() else None)

# =============================================================================
# CONFIGURATION
# =============================================================================
@dataclass
class Config:
    """Central configuration for FinBot Fine-tuning Experiment."""
    HF_TOKEN: str = os.getenv("HF_TOKEN", "")
    GEMINI_API_KEY: str = os.getenv("GEMINI_API_KEY", "")
    GEMINI_MODEL: str = "gemini-2.5-pro"
    
    # Model (optimized for 10GB VRAM)
    MODEL_NAME: str = "unsloth/Qwen2.5-3B-Instruct"
    MAX_SEQ_LENGTH: int = 2048
    LOAD_IN_4BIT: bool = True
    
    # LoRA
    LORA_R: int = 16
    LORA_ALPHA: int = 16
    LORA_DROPOUT: float = 0.0
    
    # Data Generation
    SYNTHETIC_SAMPLES: int = 150
    SAMPLES_PER_TOOL: int = 38
    
    # Training
    SFT_EPOCHS: int = 3
    SFT_LEARNING_RATE: float = 2e-5
    SFT_BATCH_SIZE: int = 4
    SFT_GRADIENT_ACCUMULATION: int = 4
    
    # Evaluation
    EVAL_SAMPLES: int = 20
    MAX_NEW_TOKENS: int = 512  # Increased for full output capture
    RANDOM_SEED: int = 42
    
    # MLflow
    MLFLOW_EXPERIMENT_NAME: str = "FinBot-Tool-Calling"
    MLFLOW_TRACKING_URI: str = f"file://{MLRUNS_DIR}"

config = Config()

print("="*70)
print("FINBOT CONFIGURATION")
print("="*70)
print(f"  Model: {config.MODEL_NAME}")
print(f"  LoRA Rank: {config.LORA_R}")
print(f"  Max New Tokens: {config.MAX_NEW_TOKENS} (no truncation)")
print(f"  HF Token: {'✓' if config.HF_TOKEN else '✗'}")
print(f"  Gemini API Key: {'✓' if config.GEMINI_API_KEY else '✗'}")
print("="*70)

FINBOT CONFIGURATION
  Model: unsloth/Qwen2.5-3B-Instruct
  LoRA Rank: 16
  Max New Tokens: 512 (no truncation)
  HF Token: ✓
  Gemini API Key: ✓


In [2]:
# =============================================================================
# MLFLOW INITIALIZATION
# =============================================================================
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri(config.MLFLOW_TRACKING_URI)
mlflow.set_experiment(config.MLFLOW_EXPERIMENT_NAME)

client = MlflowClient()
experiment = client.get_experiment_by_name(config.MLFLOW_EXPERIMENT_NAME)

print("✓ MLflow initialized")
print(f"  Experiment: {config.MLFLOW_EXPERIMENT_NAME}")
print(f"  View at: http://localhost:5000")

✓ MLflow initialized
  Experiment: FinBot-Tool-Calling
  View at: http://localhost:5000


In [3]:
# =============================================================================
# GEMINI CLIENT (for LLM-as-Judge and Synthetic Data Generation)
# =============================================================================
import google.genai as genai

gemini = genai.Client(api_key=config.GEMINI_API_KEY)
print(f"✓ Gemini client initialized with {config.GEMINI_MODEL}")

✓ Gemini client initialized with gemini-2.5-pro


In [4]:
# =============================================================================
# EXPANDED FINBOT TOOL DEFINITIONS (10 Tools for Complexity)
# =============================================================================
# We expand from 4 to 10 tools to:
# 1. Create tool selection ambiguity (e.g., earnings vs historical data)
# 2. Enable multi-step queries requiring multiple tools
# 3. Test domain-specific financial reasoning
# =============================================================================

VALID_PERIODS = ["1d", "5d", "1mo", "3mo", "6mo", "1y", "2y", "5y", "10y", "ytd", "max"]
VALID_METRICS = ["price", "volume", "market_cap", "pe_ratio", "dividend_yield"]

# Tools in OpenAI-compatible JSON Schema format for Qwen's native tool calling
FINBOT_TOOLS = [
    # === PRICE TOOLS ===
    {
        "type": "function",
        "function": {
            "name": "get_current_price",
            "description": "Get the latest real-time stock price for a ticker symbol. Use for current/today's price queries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "Stock ticker symbol in uppercase (e.g., AAPL, MSFT)"}
                },
                "required": ["ticker"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_historical_data",
            "description": "Get historical OHLCV price data over a time period. Use for past performance, price trends, or chart data.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "Stock ticker symbol in uppercase"},
                    "period": {
                        "type": "string",
                        "enum": ["1d", "5d", "1mo", "3mo", "6mo", "1y", "2y", "5y", "10y", "ytd", "max"],
                        "description": "Time period for historical data"
                    }
                },
                "required": ["ticker", "period"]
            }
        }
    },
    # === FUNDAMENTALS TOOLS ===
    {
        "type": "function",
        "function": {
            "name": "get_company_profile",
            "description": "Get company fundamentals: sector, industry, business summary, market cap, P/E ratio. Use for 'what does X do' or company info queries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "Stock ticker symbol in uppercase"}
                },
                "required": ["ticker"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_earnings_history",
            "description": "Get historical earnings per share (EPS) data, including actual vs estimated and surprises. Use for earnings performance queries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "Stock ticker symbol in uppercase"}
                },
                "required": ["ticker"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_dividend_history",
            "description": "Get dividend payment history including dates and amounts. Use for dividend/income investing queries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "Stock ticker symbol in uppercase"}
                },
                "required": ["ticker"]
            }
        }
    },
    # === ANALYSIS TOOLS ===
    {
        "type": "function",
        "function": {
            "name": "get_analyst_recommendations",
            "description": "Get analyst ratings, buy/sell/hold recommendations, and price targets. Use for analyst opinion queries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "Stock ticker symbol in uppercase"}
                },
                "required": ["ticker"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_news",
            "description": "Get latest news articles and headlines about a stock. Use for recent news or sentiment queries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "Stock ticker symbol in uppercase"}
                },
                "required": ["ticker"]
            }
        }
    },
    # === OWNERSHIP TOOLS ===
    {
        "type": "function",
        "function": {
            "name": "get_insider_trades",
            "description": "Get recent insider buying/selling transactions by executives and directors. Use for insider activity queries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "Stock ticker symbol in uppercase"}
                },
                "required": ["ticker"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_institutional_holders",
            "description": "Get list of institutional investors (funds, banks) holding the stock. Use for ownership/who owns queries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "Stock ticker symbol in uppercase"}
                },
                "required": ["ticker"]
            }
        }
    },
    # === COMPARATIVE TOOL ===
    {
        "type": "function",
        "function": {
            "name": "compare_stocks",
            "description": "Compare multiple stocks on a specific metric. Use for comparison or 'which is better' queries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "tickers": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "List of ticker symbols to compare (e.g., ['AAPL', 'MSFT', 'GOOGL'])"
                    },
                    "metric": {
                        "type": "string",
                        "enum": ["price", "volume", "market_cap", "pe_ratio", "dividend_yield"],
                        "description": "Metric to compare across stocks"
                    }
                },
                "required": ["tickers", "metric"]
            }
        }
    }
]

# Valid tool names for validation
VALID_TOOL_NAMES = [t["function"]["name"] for t in FINBOT_TOOLS]

# Tool categories for analysis
TOOL_CATEGORIES = {
    "price": ["get_current_price", "get_historical_data"],
    "fundamentals": ["get_company_profile", "get_earnings_history", "get_dividend_history"],
    "analysis": ["get_analyst_recommendations", "get_news"],
    "ownership": ["get_insider_trades", "get_institutional_holders"],
    "comparative": ["compare_stocks"]
}

SAMPLE_TICKERS = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "TSLA", "JPM",
    "V", "JNJ", "WMT", "PG", "MA", "HD", "DIS", "NFLX", "PYPL",
    "INTC", "AMD", "CRM", "ORCL", "CSCO", "IBM", "GS", "BAC",
]

# System prompt for context (tools will be injected by apply_chat_template)
FINBOT_SYSTEM_PROMPT = """You are FinBot, a precise financial market analyst assistant.
Your role is to select the appropriate tool(s) and provide correct parameters based on user queries.

For MULTI-STEP queries requiring multiple tools, output each tool call on a separate line.

Period mapping guide for get_historical_data:
- "last year" / "past year" / "12 months" → period="1y"
- "last 6 months" / "past 6 months" / "half year" → period="6mo"
- "last quarter" / "3 months" / "this quarter" → period="3mo"
- "last month" / "past month" → period="1mo"
- "last week" / "past 5 days" → period="5d"
- "year to date" / "ytd" → period="ytd"
- "5 years" / "past 5 years" → period="5y"
- "since COVID" / "pandemic" → period="5y"
- "since ChatGPT" / "AI boom" → period="2y"

Always use uppercase ticker symbols (AAPL, not aapl).
For compare_stocks, provide tickers as a list: ["AAPL", "MSFT"]."""

print("✓ Expanded tool definitions loaded (10 tools)")
print(f"  Tools: {VALID_TOOL_NAMES}")
print(f"  Categories: {list(TOOL_CATEGORIES.keys())}")
print(f"  Valid periods: {VALID_PERIODS}")
print(f"  Valid metrics: {VALID_METRICS}")

✓ Expanded tool definitions loaded (10 tools)
  Tools: ['get_current_price', 'get_historical_data', 'get_company_profile', 'get_earnings_history', 'get_dividend_history', 'get_analyst_recommendations', 'get_news', 'get_insider_trades', 'get_institutional_holders', 'compare_stocks']
  Categories: ['price', 'fundamentals', 'analysis', 'ownership', 'comparative']
  Valid periods: ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
  Valid metrics: ['price', 'volume', 'market_cap', 'pe_ratio', 'dividend_yield']


In [5]:
# =============================================================================
# HELPER FUNCTIONS FOR TOOL CALL FORMATTING
# =============================================================================

def make_tool_call(name: str, **kwargs) -> str:
    """Create a native <tool_call> formatted string."""
    return f'<tool_call>\n{json.dumps({"name": name, "arguments": kwargs})}\n</tool_call>'

def make_tool_sequence(calls: List[Dict]) -> str:
    """Create a sequence of tool calls for multi-step queries."""
    return "\n".join([
        f'<tool_call>\n{json.dumps({"name": c["name"], "arguments": c["arguments"]})}\n</tool_call>'
        for c in calls
    ])

def parse_tool_sequence(text: str) -> List[Dict]:
    """Parse multiple tool calls from model output."""
    import re
    pattern = r'<tool_call>\s*(\{.*?\})\s*</tool_call>'
    matches = re.findall(pattern, text, re.DOTALL)
    results = []
    for match in matches:
        try:
            data = json.loads(match)
            results.append({"name": data.get("name"), "arguments": data.get("arguments", {})})
        except:
            pass
    return results

print("✓ Tool call helper functions defined")

✓ Tool call helper functions defined


## Section 2: Load Base Model

In [6]:
# =============================================================================
# LOAD BASE MODEL
# =============================================================================
from unsloth import FastLanguageModel

print(f"Loading {config.MODEL_NAME}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    config.MODEL_NAME,
    max_seq_length=config.MAX_SEQ_LENGTH,
    load_in_4bit=config.LOAD_IN_4BIT,
    fast_inference=False,
    device_map="cuda:0",
)

print(f"✓ Model loaded: {type(model).__name__}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading unsloth/Qwen2.5-3B-Instruct...
==((====))==  Unsloth 2025.12.9: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GB10. Num GPUs = 1. Max memory: 119.635 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0a0+b558c986e8.nv25.11. CUDA: 12.1. CUDA Toolkit: 13.0. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

✓ Model loaded: Qwen2ForCausalLM


## Section 3: Define Comprehensive Evaluation Functions

Evaluation functions with **FULL verbose logging** - no truncation of model outputs.

In [7]:
# =============================================================================
# PARSING FUNCTIONS FOR NATIVE TOOL CALLING (<tool_call> format)
# =============================================================================

def extract_tool_calls(text: str) -> List[Dict]:
    """
    Extract Qwen-style tool calls from model output.
    
    Qwen outputs tool calls in <tool_call> tags with JSON content:
    <tool_call>
    {"name": "function_name", "arguments": {"param": "value"}}
    </tool_call>
    
    Returns list of parsed tool call dicts.
    """
    tool_calls = []
    
    if not text or not isinstance(text, str):
        return tool_calls
    
    text = text.strip()
    
    # Pattern 1: Native <tool_call> tags (primary format)
    tool_call_pattern = r'<tool_call>\s*(\{.*?\})\s*</tool_call>'
    matches = re.findall(tool_call_pattern, text, re.DOTALL)
    
    for match in matches:
        try:
            call_data = json.loads(match)
            name = call_data.get("name", "")
            args = call_data.get("arguments", {})
            if isinstance(args, str):
                args = json.loads(args)
            tool_calls.append({"name": name, "arguments": args, "raw_json": match})
        except (json.JSONDecodeError, TypeError) as e:
            tool_calls.append({"name": None, "arguments": {}, "parse_error": str(e), "raw_json": match})
    
    if tool_calls:
        return tool_calls
    
    # Pattern 2: Raw JSON with name and arguments (fallback)
    json_pattern = r'\{[^{}]*"name"\s*:\s*"(\w+)"[^{}]*"arguments"\s*:\s*(\{[^{}]*\})[^{}]*\}'
    matches = re.findall(json_pattern, text, re.DOTALL)
    
    for name, args_str in matches:
        try:
            args = json.loads(args_str)
            tool_calls.append({"name": name, "arguments": args, "raw_json": f'{{"name":"{name}","arguments":{args_str}}}'})
        except json.JSONDecodeError:
            tool_calls.append({"name": name, "arguments": {}, "parse_error": "Invalid JSON args"})
    
    return tool_calls


def parse_tool_call(text: str) -> Dict[str, Any]:
    """
    Parse tool call from model output into standardized format.
    Returns dict with: tool_name, ticker, period, raw, arguments, parse_success
    """
    result = {
        "tool_name": None, 
        "ticker": None, 
        "period": None, 
        "raw": text,
        "arguments": {},
        "parse_success": False,
        "parse_method": None
    }
    
    if not text or not isinstance(text, str):
        return result
    
    # Try native tool call extraction first
    tool_calls = extract_tool_calls(text)
    
    if tool_calls:
        call = tool_calls[0]  # Take first tool call
        result["tool_name"] = call.get("name")
        result["arguments"] = call.get("arguments", {})
        result["ticker"] = result["arguments"].get("ticker", "").upper() if result["arguments"].get("ticker") else None
        result["period"] = result["arguments"].get("period")
        result["parse_success"] = result["tool_name"] is not None
        result["parse_method"] = "native_tool_call"
        return result
    
    # Fallback: Try to parse old-style function call format (for backward compatibility)
    # e.g., get_current_price(ticker="AAPL")
    tool_match = re.match(r'(\w+)\s*\(', text.strip())
    if tool_match:
        result["tool_name"] = tool_match.group(1)
        result["parse_method"] = "legacy_function_call"
        
        # Extract ticker
        ticker_match = re.search(r'ticker\s*=\s*["\']([^"\']+)["\']', text)
        if ticker_match:
            result["ticker"] = ticker_match.group(1).upper()
        
        # Extract period
        period_match = re.search(r'period\s*=\s*["\']([^"\']+)["\']', text)
        if period_match:
            result["period"] = period_match.group(1)
        
        result["parse_success"] = True
    
    return result


def format_tool_call_for_display(parsed: Dict) -> str:
    """Format parsed tool call for human-readable display."""
    if not parsed.get("parse_success"):
        return f"[PARSE FAILED] Raw: {parsed.get('raw', '')[:100]}..."
    
    args_str = ", ".join(f'{k}="{v}"' for k, v in parsed.get("arguments", {}).items())
    return f'{parsed["tool_name"]}({args_str})'


def compute_metrics(predictions: List[str], ground_truths: List[str], tools: List[str]) -> Dict:
    """Compute accuracy metrics from predictions vs ground truth."""
    results = {
        "tool_correct": 0, "ticker_correct": 0, "period_correct": 0, 
        "period_total": 0, "overall_correct": 0, "parse_success": 0
    }
    
    for pred, gt, tool in zip(predictions, ground_truths, tools):
        pred_p = parse_tool_call(pred)
        gt_p = parse_tool_call(gt)
        
        if pred_p["parse_success"]:
            results["parse_success"] += 1
        
        tool_ok = pred_p["tool_name"] == gt_p["tool_name"]
        ticker_ok = pred_p["ticker"] == gt_p["ticker"]
        
        if tool_ok: results["tool_correct"] += 1
        if ticker_ok: results["ticker_correct"] += 1
        
        if tool == "get_historical_data":
            results["period_total"] += 1
            if pred_p["period"] == gt_p["period"]:
                results["period_correct"] += 1
        
        if tool_ok and ticker_ok:
            if tool == "get_historical_data":
                if pred_p["period"] == gt_p["period"]:
                    results["overall_correct"] += 1
            else:
                results["overall_correct"] += 1
    
    n = len(predictions)
    return {
        "tool_name_accuracy": results["tool_correct"] / n,
        "ticker_accuracy": results["ticker_correct"] / n,
        "period_accuracy": results["period_correct"] / results["period_total"] if results["period_total"] > 0 else 1.0,
        "overall_accuracy": results["overall_correct"] / n,
        "parse_success_rate": results["parse_success"] / n,
    }


print("✓ Parsing functions defined (native <tool_call> format + legacy fallback)")

✓ Parsing functions defined (native <tool_call> format + legacy fallback)


In [8]:
# =============================================================================
# EVALUATION METRICS FOR THREE-TIER QUERIES
# =============================================================================
# Metrics:
# 1. Tool Selection Accuracy: Are the correct tools selected?
# 2. Sequence Accuracy: For multi-step, is the order correct?
# 3. Parameter Accuracy: Are ticker, period, etc. correct?
# 4. Overall Pass Rate: Complete correctness
# =============================================================================

SCORE_MAP = {"perfect": 1.0, "correct": 0.8, "partial": 0.4, "wrong": 0.0}

def compute_tool_metrics(predicted_tools: List[Dict], expected_tools: List[Dict]) -> Dict:
    """
    Compute detailed metrics comparing predicted vs expected tool calls.
    
    Returns dict with:
    - tool_selection_accuracy: % of expected tools that were called
    - sequence_accuracy: 1.0 if order matches, 0.0 otherwise  
    - parameter_accuracy: % of parameters that match
    - overall_pass: True if all tools and params match
    """
    if not expected_tools:
        return {"tool_selection_accuracy": 1.0, "sequence_accuracy": 1.0, 
                "parameter_accuracy": 1.0, "overall_pass": True}
    
    if not predicted_tools:
        return {"tool_selection_accuracy": 0.0, "sequence_accuracy": 0.0,
                "parameter_accuracy": 0.0, "overall_pass": False}
    
    # Tool selection: which expected tools were called?
    expected_names = [t["name"] for t in expected_tools]
    predicted_names = [t["name"] for t in predicted_tools]
    
    tools_found = sum(1 for name in expected_names if name in predicted_names)
    tool_selection_acc = tools_found / len(expected_names)
    
    # Sequence accuracy: do the tools appear in the same order?
    sequence_acc = 1.0 if predicted_names[:len(expected_names)] == expected_names else 0.0
    
    # Parameter accuracy: for matching tools, do params match?
    param_matches = 0
    param_total = 0
    
    for exp_tool in expected_tools:
        # Find matching predicted tool
        matching = [p for p in predicted_tools if p["name"] == exp_tool["name"]]
        if matching:
            pred_tool = matching[0]
            exp_args = exp_tool.get("arguments", {})
            pred_args = pred_tool.get("arguments", {})
            
            for key, exp_val in exp_args.items():
                param_total += 1
                pred_val = pred_args.get(key)
                
                # Handle case-insensitive ticker comparison
                if key == "ticker" and isinstance(exp_val, str) and isinstance(pred_val, str):
                    if exp_val.upper() == pred_val.upper():
                        param_matches += 1
                elif key == "tickers" and isinstance(exp_val, list) and isinstance(pred_val, list):
                    if set(v.upper() for v in exp_val) == set(v.upper() for v in pred_val):
                        param_matches += 1
                elif exp_val == pred_val:
                    param_matches += 1
    
    param_acc = param_matches / param_total if param_total > 0 else 1.0
    
    # Overall pass: all tools correct with correct params
    overall = (tool_selection_acc == 1.0 and param_acc == 1.0)
    
    return {
        "tool_selection_accuracy": tool_selection_acc,
        "sequence_accuracy": sequence_acc,
        "parameter_accuracy": param_acc,
        "overall_pass": overall,
        "expected_tools": expected_names,
        "predicted_tools": predicted_names,
    }


JUDGE_PROMPT_TEMPLATE = """You are evaluating a financial assistant's tool selection for a {category} query.

Query Type: {category}
User Query: {query}
Expected Tool Calls: {expected}
Model's Output: {predicted}

For {category} queries, evaluate:
- DIRECT: Single tool with correct parameters
- MULTI_STEP: Correct tools in logical sequence
- DOMAIN_SPECIFIC: Correct interpretation of implicit parameters

Rate the model's response using ONLY one of these categories:
- 'perfect': All expected tools called with correct parameters in correct order
- 'correct': Right tools selected, minor parameter variations (e.g., period off by one level)
- 'partial': Some tools correct but missing others, or significant parameter errors
- 'wrong': Wrong tools selected or completely incorrect interpretation

Respond in this exact JSON format:
{{"category": "<perfect|correct|partial|wrong>", "rationale": "<your explanation>"}}
"""

def judge_response(query: str, expected: str, predicted: str, category: str = "direct") -> Dict:
    """Evaluate tool call quality using Gemini as judge."""
    try:
        prompt = JUDGE_PROMPT_TEMPLATE.format(
            query=query,
            expected=expected,
            predicted=predicted,
            category=category
        )
        
        response = gemini.models.generate_content(
            model=config.GEMINI_MODEL,
            contents=prompt,
        )
        
        text = response.text.strip()
        
        # Parse JSON response
        if "```json" in text:
            text = text.split("```json")[1].split("```")[0]
        elif "```" in text:
            text = text.split("```")[1].split("```")[0]
        
        result = json.loads(text)
        cat = result.get("category", "wrong").lower()
        rationale = result.get("rationale", "No rationale provided")
        
        return {
            "score": SCORE_MAP.get(cat, 0),
            "category": cat,
            "rationale": rationale
        }
        
    except Exception as e:
        return {"score": 0, "category": "error", "rationale": str(e)}

print("✓ Three-tier evaluation metrics defined")
print("  - Tool Selection Accuracy")
print("  - Sequence Accuracy (for multi-step)")
print("  - Parameter Accuracy")
print("  - Overall Pass Rate")

✓ Three-tier evaluation metrics defined
  - Tool Selection Accuracy
  - Sequence Accuracy (for multi-step)
  - Parameter Accuracy
  - Overall Pass Rate


In [9]:
# =============================================================================
# THREE-TIER EVALUATION WITH COMPREHENSIVE LOGGING
# =============================================================================
# Evaluates: Direct, Multi-Step, and Domain-Specific queries
# Metrics: Tool Selection, Sequence Accuracy, Parameter Accuracy, Overall Pass
# =============================================================================

def generate_response(model, tokenizer, query: str, return_prompt: bool = False) -> str:
    """
    Generate model response using NATIVE tool calling.
    Uses apply_chat_template with tools= parameter for proper Qwen tool calling.
    """
    messages = [
        {"role": "system", "content": FINBOT_SYSTEM_PROMPT},
        {"role": "user", "content": query},
    ]
    
    # Apply chat template WITH TOOLS for native function calling
    try:
        prompt = tokenizer.apply_chat_template(
            messages,
            tools=FINBOT_TOOLS,  # Pass tools for native function calling!
            tokenize=False,
            add_generation_prompt=True,
        )
    except Exception as e:
        # Fallback: some tokenizers don't support tools parameter
        print(f"⚠️ apply_chat_template with tools failed: {e}")
        prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, 
                       max_length=config.MAX_SEQ_LENGTH - config.MAX_NEW_TOKENS).to("cuda")
    
    input_len = inputs["input_ids"].shape[1]
    outputs = model.generate(
        **inputs, 
        max_new_tokens=config.MAX_NEW_TOKENS, 
        do_sample=False, 
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True).strip()
    
    if return_prompt:
        return response, prompt
    return response


def run_three_tier_evaluation(model, tokenizer, queries: List[Dict], stage: str, 
                              run_name: str) -> tuple:
    """
    Run THREE-TIER evaluation with comprehensive logging.
    
    Evaluates:
    - DIRECT queries (single tool)
    - MULTI_STEP queries (tool sequences)
    - DOMAIN_SPECIFIC queries (implicit reasoning)
    
    Metrics:
    - Tool Selection Accuracy
    - Sequence Accuracy (for multi-step)
    - Parameter Accuracy
    - Overall Pass Rate
    """
    results = []
    
    # Category-specific metrics
    category_metrics = {
        "direct": {"total": 0, "pass": 0, "tool_acc": [], "param_acc": []},
        "multi_step": {"total": 0, "pass": 0, "tool_acc": [], "seq_acc": [], "param_acc": []},
        "domain_specific": {"total": 0, "pass": 0, "tool_acc": [], "param_acc": []}
    }
    
    with mlflow.start_run(run_name=run_name):
        mlflow.log_params({
            "model_name": config.MODEL_NAME,
            "stage": stage,
            "eval_samples": len(queries),
            "tool_calling_format": "native_qwen",
            "num_tools": len(VALID_TOOL_NAMES),
        })
        
        print("\n" + "█"*95)
        print(f"█  {stage.upper()} MODEL - THREE-TIER EVALUATION")
        print("█" + "─"*93 + "█")
        print(f"█  Samples: {len(queries)} | Categories: direct, multi_step, domain_specific")
        print("█"*95)
        
        for i, q in enumerate(queries):
            query = q["query"]
            expected = q.get("expected", "")
            expected_tools = q.get("expected_tools", [])
            category = q.get("category", "direct")
            num_expected = q.get("num_tools", 1)
            
            # Generate response
            start = time.time()
            predicted, prompt_used = generate_response(model, tokenizer, query, return_prompt=True)
            latency = time.time() - start
            
            # Parse predicted tool calls (may be multiple)
            predicted_tools = parse_tool_sequence(predicted)
            
            # Compute metrics
            metrics = compute_tool_metrics(predicted_tools, expected_tools)
            
            # LLM Judge
            judge = judge_response(query, expected, predicted, category)
            
            # Update category metrics
            cat_m = category_metrics.get(category, category_metrics["direct"])
            cat_m["total"] += 1
            if metrics["overall_pass"]:
                cat_m["pass"] += 1
            cat_m["tool_acc"].append(metrics["tool_selection_accuracy"])
            cat_m["param_acc"].append(metrics["parameter_accuracy"])
            if category == "multi_step":
                cat_m["seq_acc"].append(metrics["sequence_accuracy"])
            
            # Store result
            result_record = {
                "test_num": i + 1,
                "query": query,
                "category": category,
                "num_expected_tools": num_expected,
                "num_predicted_tools": len(predicted_tools),
                "expected_tools": [t["name"] for t in expected_tools],
                "predicted_tools": [t.get("name") for t in predicted_tools],
                "expected_raw": expected,
                "predicted_raw": predicted,
                "tool_selection_accuracy": metrics["tool_selection_accuracy"],
                "sequence_accuracy": metrics["sequence_accuracy"],
                "parameter_accuracy": metrics["parameter_accuracy"],
                "overall_pass": metrics["overall_pass"],
                "judge_score": judge["score"],
                "judge_category": judge["category"],
                "judge_rationale": judge["rationale"],
                "latency": latency,
            }
            results.append(result_record)
            
            # ═══════════════════════════════════════════════════════════════════════════
            # VERBOSE OUTPUT
            # ═══════════════════════════════════════════════════════════════════════════
            status = "✅ PASS" if metrics["overall_pass"] else "❌ FAIL"
            cat_emoji = {"direct": "🎯", "multi_step": "🔗", "domain_specific": "🧠"}.get(category, "📝")
            
            print(f"\n{'═'*95}")
            print(f"TEST {i+1}/{len(queries)}: {status} | {cat_emoji} {category.upper()}")
            print(f"{'═'*95}")
            
            print(f"\n📝 QUERY: \"{query}\"")
            
            print(f"\n🎯 EXPECTED ({num_expected} tool{'s' if num_expected > 1 else ''}):")
            for t in expected_tools:
                print(f"   • {t['name']}({', '.join(f'{k}={v}' for k,v in t.get('arguments',{}).items())})")
            
            print(f"\n🤖 MODEL OUTPUT (FULL):")
            print(f"   ┌{'─'*91}┐")
            for line in predicted.split('\n'):
                while len(line) > 89:
                    print(f"   │ {line[:89]} │")
                    line = line[89:]
                print(f"   │ {line:<89} │")
            print(f"   └{'─'*91}┘")
            
            print(f"\n🔍 PARSED TOOL CALLS ({len(predicted_tools)} found):")
            for j, t in enumerate(predicted_tools):
                print(f"   {j+1}. {t.get('name', 'UNKNOWN')}({', '.join(f'{k}={v}' for k,v in t.get('arguments',{}).items())})")
            if not predicted_tools:
                print(f"   ⚠️ No valid tool calls parsed!")
            
            print(f"\n📊 METRICS:")
            print(f"   Tool Selection: {metrics['tool_selection_accuracy']*100:5.1f}%  {'✓' if metrics['tool_selection_accuracy']==1 else '✗'}")
            if category == "multi_step":
                print(f"   Sequence:       {metrics['sequence_accuracy']*100:5.1f}%  {'✓' if metrics['sequence_accuracy']==1 else '✗'}")
            print(f"   Parameters:     {metrics['parameter_accuracy']*100:5.1f}%  {'✓' if metrics['parameter_accuracy']==1 else '✗'}")
            print(f"   Overall Pass:   {'YES ✓' if metrics['overall_pass'] else 'NO ✗'}")
            
            print(f"\n⚖️ LLM JUDGE: {judge['category'].upper()} ({judge['score']:.1f})")
            print(f"   {judge['rationale'][:150]}{'...' if len(judge['rationale'])>150 else ''}")
            
            print(f"\n⏱️ Latency: {latency:.3f}s")
        
        # Aggregate metrics
        df = pd.DataFrame(results)
        
        # Overall metrics
        overall_pass_rate = df["overall_pass"].mean()
        avg_tool_acc = df["tool_selection_accuracy"].mean()
        avg_param_acc = df["parameter_accuracy"].mean()
        avg_judge = df["judge_score"].mean()
        
        # Log to MLflow
        mlflow.log_metrics({
            f"{stage}_overall_pass_rate": overall_pass_rate,
            f"{stage}_tool_selection_accuracy": avg_tool_acc,
            f"{stage}_parameter_accuracy": avg_param_acc,
            f"{stage}_judge_score_avg": avg_judge,
            f"{stage}_avg_latency": df["latency"].mean(),
        })
        
        # Log per-category metrics
        for cat, m in category_metrics.items():
            if m["total"] > 0:
                mlflow.log_metrics({
                    f"{stage}_{cat}_pass_rate": m["pass"] / m["total"],
                    f"{stage}_{cat}_tool_acc": sum(m["tool_acc"]) / len(m["tool_acc"]) if m["tool_acc"] else 0,
                    f"{stage}_{cat}_param_acc": sum(m["param_acc"]) / len(m["param_acc"]) if m["param_acc"] else 0,
                })
        
        # Save artifacts
        csv_file = f"{stage}_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(csv_file, index=False)
        mlflow.log_artifact(csv_file)
        
        json_file = f"{stage}_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(json_file, 'w') as f:
            json.dump(results, f, indent=2)
        mlflow.log_artifact(json_file)
        
        # Summary
        print(f"\n{'█'*95}")
        print(f"█  {stage.upper()} MODEL - EVALUATION SUMMARY")
        print(f"{'█'*95}")
        
        print(f"\n📊 OVERALL METRICS:")
        print(f"   Overall Pass Rate:      {overall_pass_rate*100:6.1f}%")
        print(f"   Tool Selection Acc:     {avg_tool_acc*100:6.1f}%")
        print(f"   Parameter Accuracy:     {avg_param_acc*100:6.1f}%")
        print(f"   LLM Judge Score:        {avg_judge*100:6.1f}%")
        
        print(f"\n📈 BY CATEGORY:")
        for cat, m in category_metrics.items():
            if m["total"] > 0:
                pass_rate = m["pass"] / m["total"] * 100
                tool_acc = sum(m["tool_acc"]) / len(m["tool_acc"]) * 100 if m["tool_acc"] else 0
                emoji = {"direct": "🎯", "multi_step": "🔗", "domain_specific": "🧠"}.get(cat, "")
                print(f"   {emoji} {cat:20s}: {m['pass']:2d}/{m['total']:2d} pass ({pass_rate:5.1f}%) | Tool Acc: {tool_acc:5.1f}%")
        
        print(f"\n🧠 LLM JUDGE BREAKDOWN:")
        for cat in ["perfect", "correct", "partial", "wrong"]:
            count = (df["judge_category"] == cat).sum()
            print(f"   {cat.capitalize():10s}: {count:2d}/{len(df)}")
        
        print(f"{'█'*95}")
        
        return df, category_metrics, {"overall_pass_rate": overall_pass_rate, "judge_avg": avg_judge}


print("✓ Three-tier evaluation function defined")
print("  Evaluates: direct, multi_step, domain_specific queries")
print("  Metrics: tool_selection, sequence, parameter, overall_pass")

✓ Three-tier evaluation function defined
  Evaluates: direct, multi_step, domain_specific queries
  Metrics: tool_selection, sequence, parameter, overall_pass


## Section 4: Generate Synthetic Dataset with Three Query Types

We use **Gemini 2.5 Pro** to generate a comprehensive synthetic dataset with three categories of queries.

### Query Categories

| Type | Description | Tool Complexity | Example |
|------|-------------|-----------------|---------|
| 🎯 **Direct** | Single tool, explicit parameters | 1 tool | "What's Apple's stock price?" |
| 🔗 **Multi-Step** | Sequential tools, comparative analysis | 2-3 tools | "Compare AAPL and MSFT performance and analyst ratings" |
| 🧠 **Domain-Specific** | Financial reasoning, implicit parameters | 1-2 tools | "How did Tesla react to its last earnings report?" |

### Dataset Split Strategy

- **Total Generated**: ~150 examples (50 per category)
- **Evaluation Set**: 12 examples per category (36 total, held out)
- **Training Set**: Remaining examples (~114 for fine-tuning)

### Data Storage

All synthetic data is saved to `data/processed/finbot_tool_calling/`:
- `synthetic_data_v2.json` - Full generated dataset
- `train_data.json` - Training split
- `eval_data.json` - Evaluation split

### Regeneration Control

Set `REGENERATE_DATA = True` in the cell below to force regeneration even if data already exists.

In [10]:
# =============================================================================
# THREE-TIER SYNTHETIC DATA GENERATION
# =============================================================================
# Categories:
# 1. DIRECT: Single tool, explicit parameters (easy)
# 2. MULTI_STEP: Sequential tool chains (medium)  
# 3. DOMAIN_SPECIFIC: Financial reasoning, implicit parameters (hard)
# =============================================================================

PERIOD_MAPPING = {
    "past day": "1d", "yesterday": "1d", "last day": "1d",
    "past 5 days": "5d", "last 5 days": "5d", "past week": "5d",
    "past month": "1mo", "last month": "1mo", "30 days": "1mo",
    "past 3 months": "3mo", "last quarter": "3mo", "this quarter": "3mo",
    "past 6 months": "6mo", "half year": "6mo", "last 6 months": "6mo",
    "past year": "1y", "last year": "1y", "12 months": "1y",
    "past 2 years": "2y", "last 2 years": "2y", "since ChatGPT": "2y",
    "past 5 years": "5y", "last 5 years": "5y", "since COVID": "5y",
    "past 10 years": "10y", "last decade": "10y",
    "year to date": "ytd", "this year so far": "ytd",
    "all time": "max", "full history": "max",
}

# Tool info lookup
TOOL_INFO = {t["function"]["name"]: t["function"] for t in FINBOT_TOOLS}

# =============================================================================
# GENERATION PROMPTS FOR EACH CATEGORY
# =============================================================================

DIRECT_PROMPT = """Generate {count} DIRECT (single-tool) queries for a financial assistant.

Available Tools:
{tools_desc}

Available Tickers: {tickers}
Valid Periods (for get_historical_data): {periods}
Valid Metrics (for compare_stocks): {metrics}

OUTPUT FORMAT (JSON array):
[
  {{
    "query": "natural language question",
    "category": "direct",
    "expected_tools": [
      {{"name": "tool_name", "arguments": {{...}}}}
    ]
  }},
  ...
]

RULES:
- Each query should use EXACTLY ONE tool
- Vary phrasing: casual, formal, short, long
- Use different tickers
- Include queries for ALL 10 tools
- For get_historical_data, use natural time phrases ("last year", "past 6 months")
- Output ONLY valid JSON, no explanation

EXAMPLES:
- "What's Apple's current stock price?" → get_current_price(ticker="AAPL")
- "Show me Tesla's earnings history" → get_earnings_history(ticker="TSLA")
- "Who are the major institutional holders of Microsoft?" → get_institutional_holders(ticker="MSFT")
"""

MULTI_STEP_PROMPT = """Generate {count} MULTI-STEP queries requiring 2-3 sequential tool calls.

Available Tools:
{tools_desc}

Available Tickers: {tickers}
Valid Periods: {periods}
Valid Metrics: {metrics}

OUTPUT FORMAT (JSON array):
[
  {{
    "query": "complex question requiring multiple tools",
    "category": "multi_step",
    "expected_tools": [
      {{"name": "first_tool", "arguments": {{...}}}},
      {{"name": "second_tool", "arguments": {{...}}}},
      ...
    ],
    "reasoning": "brief explanation of why these tools in this order"
  }},
  ...
]

RULES:
- Each query MUST require 2-3 tools to fully answer
- Tools should be called in logical order
- Include comparative queries (compare_stocks + other tools)
- Include research-style queries (profile + news + recommendations)
- Vary the tool combinations

EXAMPLE PATTERNS:
1. Price + Analyst: "Is AAPL a good buy at current price?" → get_current_price + get_analyst_recommendations
2. Compare + History: "Which performed better this year, NVDA or AMD?" → compare_stocks + get_historical_data (x2)
3. Research: "Give me a full overview of Tesla" → get_company_profile + get_news + get_analyst_recommendations
4. Earnings + Price: "How did Microsoft react to its last earnings?" → get_earnings_history + get_historical_data
5. Ownership: "Are insiders buying or selling Apple, and what do institutions think?" → get_insider_trades + get_institutional_holders
"""

DOMAIN_SPECIFIC_PROMPT = """Generate {count} DOMAIN-SPECIFIC queries requiring financial reasoning.

Available Tools:
{tools_desc}

Available Tickers: {tickers}
Valid Periods: {periods}

OUTPUT FORMAT (JSON array):
[
  {{
    "query": "question requiring financial domain knowledge",
    "category": "domain_specific",
    "expected_tools": [
      {{"name": "tool_name", "arguments": {{...}}}}
    ],
    "reasoning": "what financial knowledge is needed to answer"
  }},
  ...
]

RULES:
- Queries should require IMPLICIT parameter inference
- Use financial terminology that maps to specific tools/periods
- Include time references that require reasoning (e.g., "since the pandemic", "after the AI boom")
- Include queries about financial concepts (P/E ratio, dividend yield, earnings beats)

EXAMPLE PATTERNS:
1. Implicit period: "How has NVDA performed since ChatGPT launched?" → get_historical_data(period="2y")
2. Financial concept: "Is Apple's P/E ratio justified?" → get_company_profile (contains P/E)
3. Event-based: "How did Tesla react to its last earnings report?" → get_historical_data(period="3mo")
4. Sector analysis: "Is Microsoft outperforming the tech sector?" → get_company_profile + get_historical_data
5. Dividend focus: "Is JNJ a good dividend stock?" → get_dividend_history + get_company_profile
6. Insider signal: "Are executives bullish on their own company (AAPL)?" → get_insider_trades
"""


def generate_category_data(category: str, count: int) -> List[Dict]:
    """Generate synthetic data for a specific category using Gemini."""
    
    # Build tools description
    tools_desc = "\n".join([
        f"- {t['function']['name']}: {t['function']['description']}"
        for t in FINBOT_TOOLS
    ])
    
    tickers = ", ".join(SAMPLE_TICKERS[:15])
    
    if category == "direct":
        prompt = DIRECT_PROMPT.format(
            count=count, tools_desc=tools_desc, tickers=tickers,
            periods=", ".join(VALID_PERIODS), metrics=", ".join(VALID_METRICS)
        )
    elif category == "multi_step":
        prompt = MULTI_STEP_PROMPT.format(
            count=count, tools_desc=tools_desc, tickers=tickers,
            periods=", ".join(VALID_PERIODS), metrics=", ".join(VALID_METRICS)
        )
    else:  # domain_specific
        prompt = DOMAIN_SPECIFIC_PROMPT.format(
            count=count, tools_desc=tools_desc, tickers=tickers,
            periods=", ".join(VALID_PERIODS)
        )
    
    try:
        response = gemini.models.generate_content(
            model=config.GEMINI_MODEL,
            contents=prompt,
        )
        text = response.text.strip()
        
        # Extract JSON
        if "```json" in text:
            text = text.split("```json")[1].split("```")[0]
        elif "```" in text:
            text = text.split("```")[1].split("```")[0]
        
        examples = json.loads(text)
        
        # Add category and format tool calls
        for ex in examples:
            ex["category"] = category
            # Convert expected_tools to tool_call string format
            if "expected_tools" in ex:
                ex["tool_call"] = make_tool_sequence(ex["expected_tools"])
                ex["num_tools"] = len(ex["expected_tools"])
        
        return examples
        
    except Exception as e:
        print(f"Error generating {category} data: {e}")
        import traceback
        traceback.print_exc()
        return []


def generate_all_synthetic_data() -> Dict[str, List[Dict]]:
    """Generate synthetic data for all three categories."""
    
    all_data = {
        "direct": [],
        "multi_step": [],
        "domain_specific": []
    }
    
    # Generate 50 examples per category
    for category in all_data.keys():
        print(f"\n{'='*60}")
        print(f"Generating {category.upper()} queries...")
        print(f"{'='*60}")
        
        # Generate in batches to avoid rate limits
        batch_size = 25
        total_needed = 50
        
        for batch in range(0, total_needed, batch_size):
            remaining = min(batch_size, total_needed - batch)
            print(f"  Batch {batch//batch_size + 1}: Generating {remaining} examples...")
            
            examples = generate_category_data(category, remaining)
            all_data[category].extend(examples)
            
            print(f"    Generated {len(examples)} examples")
            time.sleep(2)  # Rate limiting
        
        print(f"  Total {category}: {len(all_data[category])} examples")
    
    return all_data


print("✓ Three-tier synthetic data generation defined")
print("  Categories: direct, multi_step, domain_specific")
print("  Target: 50 examples per category = 150 total")

✓ Three-tier synthetic data generation defined
  Categories: direct, multi_step, domain_specific
  Target: 50 examples per category = 150 total


In [11]:
# =============================================================================
# GENERATE OR LOAD SYNTHETIC DATA + TRAIN/EVAL SPLIT
# =============================================================================

# ┌─────────────────────────────────────────────────────────────────────────────┐
# │ CONFIGURATION: Set to True to force regeneration of synthetic data         │
# └─────────────────────────────────────────────────────────────────────────────┘
REGENERATE_DATA = False  # Set to True to regenerate even if data exists

# Data storage paths
SYNTHETIC_DATA_DIR = DATA_DIR / "processed" / "finbot_tool_calling"
SYNTHETIC_DATA_DIR.mkdir(parents=True, exist_ok=True)

SYNTHETIC_DATA_FILE = SYNTHETIC_DATA_DIR / "synthetic_data_v2.json"
TRAIN_DATA_FILE = SYNTHETIC_DATA_DIR / "train_data.json"
EVAL_DATA_FILE = SYNTHETIC_DATA_DIR / "eval_data.json"

print(f"📁 Synthetic data directory: {SYNTHETIC_DATA_DIR}")

# Check if we need to generate or load
should_generate = REGENERATE_DATA or not SYNTHETIC_DATA_FILE.exists()

if should_generate:
    if REGENERATE_DATA:
        print("\n⚠️ REGENERATE_DATA=True: Forcing regeneration of synthetic data...")
    else:
        print("\n📝 No existing data found. Generating new synthetic data...")
    
    print("="*80)
    synthetic_data = generate_all_synthetic_data()
    
    # Save to file
    with open(SYNTHETIC_DATA_FILE, "w") as f:
        json.dump(synthetic_data, f, indent=2)
    print(f"\n✓ Saved synthetic data to: {SYNTHETIC_DATA_FILE}")
    
    # =========================================================================
    # SHOW FULL SAMPLE RECORDS (NOT TRUNCATED)
    # =========================================================================
    print("\n" + "="*80)
    print("SAMPLE GENERATED RECORDS (FULL - NOT TRUNCATED)")
    print("="*80)
    
    for category in ["direct", "multi_step", "domain_specific"]:
        examples = synthetic_data.get(category, [])
        if examples:
            print(f"\n{'─'*80}")
            print(f"📌 {category.upper()} - Sample Record #{1}")
            print(f"{'─'*80}")
            
            sample = examples[0]
            print(f"\n🔹 Query:")
            print(f"   {sample['query']}")
            
            print(f"\n🔹 Expected Tools ({len(sample.get('expected_tools', []))}):")
            for j, tool in enumerate(sample.get('expected_tools', []), 1):
                print(f"   {j}. {tool['name']}")
                for k, v in tool.get('arguments', {}).items():
                    print(f"      └─ {k}: {v}")
            
            print(f"\n🔹 Tool Call (raw):")
            tool_call = sample.get('tool_call', '')
            for line in tool_call.split('\n'):
                print(f"   {line}")
            
            if sample.get('reasoning'):
                print(f"\n🔹 Reasoning:")
                print(f"   {sample['reasoning']}")
            
            # Show a second sample if available
            if len(examples) > 1:
                print(f"\n{'─'*80}")
                print(f"📌 {category.upper()} - Sample Record #{2}")
                print(f"{'─'*80}")
                
                sample2 = examples[1]
                print(f"\n🔹 Query:")
                print(f"   {sample2['query']}")
                
                print(f"\n🔹 Expected Tools ({len(sample2.get('expected_tools', []))}):")
                for j, tool in enumerate(sample2.get('expected_tools', []), 1):
                    print(f"   {j}. {tool['name']}")
                    for k, v in tool.get('arguments', {}).items():
                        print(f"      └─ {k}: {v}")
                
                print(f"\n🔹 Tool Call (raw):")
                tool_call2 = sample2.get('tool_call', '')
                for line in tool_call2.split('\n'):
                    print(f"   {line}")

else:
    print(f"\n✓ Loading existing synthetic data from: {SYNTHETIC_DATA_FILE}")
    with open(SYNTHETIC_DATA_FILE) as f:
        synthetic_data = json.load(f)
    print(f"  (Set REGENERATE_DATA=True to force regeneration)")

# =============================================================================
# TRAIN/EVAL SPLIT: 12 eval samples per category, rest for training
# =============================================================================

EVAL_PER_CATEGORY = 12  # Hold out 12 per category for evaluation

train_data = {"direct": [], "multi_step": [], "domain_specific": []}
eval_data = {"direct": [], "multi_step": [], "domain_specific": []}

for category in ["direct", "multi_step", "domain_specific"]:
    examples = synthetic_data.get(category, [])
    random.shuffle(examples)
    
    # Split: first EVAL_PER_CATEGORY for eval, rest for training
    eval_data[category] = examples[:EVAL_PER_CATEGORY]
    train_data[category] = examples[EVAL_PER_CATEGORY:]

# Save train/eval splits
with open(TRAIN_DATA_FILE, "w") as f:
    json.dump(train_data, f, indent=2)
with open(EVAL_DATA_FILE, "w") as f:
    json.dump(eval_data, f, indent=2)

# Flatten for training
train_flat = []
for category, examples in train_data.items():
    train_flat.extend(examples)

# Create evaluation queries in the expected format
EVAL_QUERIES = []
for category, examples in eval_data.items():
    for ex in examples:
        EVAL_QUERIES.append({
            "query": ex["query"],
            "category": category,
            "expected_tools": ex.get("expected_tools", []),
            "expected": ex.get("tool_call", ""),
            "num_tools": ex.get("num_tools", 1),
            "reasoning": ex.get("reasoning", "")
        })

# Summary
print(f"\n{'='*80}")
print("DATASET SPLIT SUMMARY")
print(f"{'='*80}")
print(f"\n📁 Data saved to: {SYNTHETIC_DATA_DIR}")
print(f"   • synthetic_data_v2.json - Full generated data")
print(f"   • train_data.json - Training split")
print(f"   • eval_data.json - Evaluation split")

print(f"\n📊 TRAINING DATA:")
for cat, examples in train_data.items():
    emoji = {"direct": "🎯", "multi_step": "🔗", "domain_specific": "🧠"}.get(cat, "")
    print(f"   {emoji} {cat}: {len(examples)} examples")
print(f"   TOTAL: {len(train_flat)} training examples")

print(f"\n📋 EVALUATION DATA:")
for cat, examples in eval_data.items():
    emoji = {"direct": "🎯", "multi_step": "🔗", "domain_specific": "🧠"}.get(cat, "")
    print(f"   {emoji} {cat}: {len(examples)} examples")
print(f"   TOTAL: {len(EVAL_QUERIES)} evaluation queries")

print(f"\n{'='*80}")
print("SAMPLE EVALUATION QUERIES (1 per category)")
print(f"{'='*80}")

for category in ["direct", "multi_step", "domain_specific"]:
    emoji = {"direct": "🎯", "multi_step": "🔗", "domain_specific": "🧠"}.get(category, "")
    print(f"\n{emoji} {category.upper()}:")
    sample = eval_data[category][0] if eval_data[category] else None
    if sample:
        print(f"   Query: {sample['query']}")
        print(f"   Tools: {[t['name'] for t in sample.get('expected_tools', [])]}")
        if sample.get('reasoning'):
            print(f"   Reasoning: {sample['reasoning']}")

📁 Synthetic data directory: /workspace/zgx-onboard/data/processed/finbot_tool_calling

📝 No existing data found. Generating new synthetic data...

Generating DIRECT queries...
  Batch 1: Generating 25 examples...
    Generated 25 examples
  Batch 2: Generating 25 examples...
    Generated 25 examples
  Total direct: 50 examples

Generating MULTI_STEP queries...
  Batch 1: Generating 25 examples...
    Generated 25 examples
  Batch 2: Generating 25 examples...
    Generated 25 examples
  Total multi_step: 50 examples

Generating DOMAIN_SPECIFIC queries...
  Batch 1: Generating 25 examples...
    Generated 25 examples
  Batch 2: Generating 25 examples...
    Generated 25 examples
  Total domain_specific: 50 examples

✓ Saved synthetic data to: /workspace/zgx-onboard/data/processed/finbot_tool_calling/synthetic_data_v2.json

SAMPLE GENERATED RECORDS (FULL - NOT TRUNCATED)

────────────────────────────────────────────────────────────────────────────────
📌 DIRECT - Sample Record #1
────────

## Section 5: Evaluate BASE Model (Before Fine-Tuning)

**Run this FIRST** to understand what issues the base model has. This informs:
- What types of errors occur
- Which tools are problematic
- What training data we need to generate

In [12]:
# =============================================================================
# EVALUATE BASE MODEL (Three-Tier Evaluation)
# =============================================================================
FastLanguageModel.for_inference(model)

print("Running BASE model evaluation across three query categories...")
print("Categories: DIRECT (single tool), MULTI_STEP (sequences), DOMAIN_SPECIFIC (reasoning)")

base_results_df, base_category_metrics, base_summary = run_three_tier_evaluation(
    model, tokenizer, EVAL_QUERIES, 
    stage="base",
    run_name=f"base-{datetime.now().strftime('%Y%m%d_%H%M%S')}"
)

Running BASE model evaluation across three query categories...
Categories: DIRECT (single tool), MULTI_STEP (sequences), DOMAIN_SPECIFIC (reasoning)

███████████████████████████████████████████████████████████████████████████████████████████████
█  BASE MODEL - THREE-TIER EVALUATION
█─────────────────────────────────────────────────────────────────────────────────────────────█
█  Samples: 36 | Categories: direct, multi_step, domain_specific
███████████████████████████████████████████████████████████████████████████████████████████████

═══════════════════════════════════════════════════════════════════════════════════════════════
TEST 1/36: ✅ PASS | 🎯 DIRECT
═══════════════════════════════════════════════════════════════════════════════════════════════

📝 QUERY: "Can you give me a summary of what Amazon does?"

🎯 EXPECTED (1 tool):
   • get_company_profile(ticker=AMZN)

🤖 MODEL OUTPUT (FULL):
   ┌──────────────────────────────────────────────────────────────────────────────────────────

In [14]:
# =============================================================================
# ANALYZE BASE MODEL FAILURES
# =============================================================================

print("\n" + "="*80)
print("BASE MODEL - FAILURE ANALYSIS")
print("="*80)

# Overall pass/fail
passed = base_results_df["overall_pass"].sum()
total = len(base_results_df)
print(f"\nOverall: {passed}/{total} passed ({passed/total*100:.1f}%)")

# Failures by category
print("\n📊 PERFORMANCE BY CATEGORY:")
for cat in base_results_df["category"].unique():
    cat_df = base_results_df[base_results_df["category"] == cat]
    cat_passed = cat_df["overall_pass"].sum()
    print(f"   {cat.upper()}: {cat_passed}/{len(cat_df)} passed ({cat_passed/len(cat_df)*100:.1f}%)")

# Accuracy breakdown
print("\n🔍 ACCURACY BREAKDOWN:")
print(f"   Tool Selection Accuracy:  {base_results_df['tool_selection_accuracy'].mean()*100:.1f}%")
print(f"   Sequence Accuracy:        {base_results_df['sequence_accuracy'].mean()*100:.1f}%")
print(f"   Parameter Accuracy:       {base_results_df['parameter_accuracy'].mean()*100:.1f}%")

# Show specific failures
print("\n❌ SPECIFIC FAILURES (for analysis):")
failures = base_results_df[~base_results_df["overall_pass"]]
for _, row in failures.head(10).iterrows():  # Show first 10 failures
    print(f"\n   [{row['category'].upper()}] Query: {row['query'][:80]}...")
    print(f"   Expected tools: {row['num_expected_tools']}, Got: {row['num_predicted_tools']}")
    print(f"   Tool Selection: {row['tool_selection_accuracy']*100:.0f}%, Sequence: {row['sequence_accuracy']*100:.0f}%, Params: {row['parameter_accuracy']*100:.0f}%")

# Judge analysis
print("\n📝 LLM JUDGE ANALYSIS:")
judge_scores = base_results_df.groupby("judge_category").size()
for cat, count in judge_scores.items():
    print(f"   {cat}: {count} ({count/total*100:.1f}%)")

print("\n" + "="*80)
print("Insights inform synthetic data generation strategy!")
print("="*80)



BASE MODEL - FAILURE ANALYSIS

Overall: 28/36 passed (77.8%)

📊 PERFORMANCE BY CATEGORY:
   DIRECT: 11/12 passed (91.7%)
   MULTI_STEP: 8/12 passed (66.7%)
   DOMAIN_SPECIFIC: 9/12 passed (75.0%)

🔍 ACCURACY BREAKDOWN:
   Tool Selection Accuracy:  87.5%
   Sequence Accuracy:        80.6%
   Parameter Accuracy:       93.1%

❌ SPECIFIC FAILURES (for analysis):

   [DIRECT] Query: Get the latest stock price for Visa....
   Expected tools: 1, Got: 0
   Tool Selection: 0%, Sequence: 0%, Params: 0%

   [MULTI_STEP] Query: Which is a better value, MSFT or GOOGL based on P/E ratio, and how has the cheap...
   Expected tools: 2, Got: 2
   Tool Selection: 100%, Sequence: 100%, Params: 75%

   [MULTI_STEP] Query: Which company is more valuable by market cap, Meta or Google? For the more valua...
   Expected tools: 2, Got: 4
   Tool Selection: 50%, Sequence: 0%, Params: 0%

   [MULTI_STEP] Query: What was Disney's biggest earnings surprise in the past two years, and how has t...
   Expected tools

## Section 6: Prepare Training Dataset

In [15]:
# =============================================================================
# PREPARE TRAINING DATASET (Three-Tier Format)
# =============================================================================

def prepare_training_dataset(data: List[Dict], tokenizer) -> Dataset:
    """
    Convert synthetic data to training format for SFT with native tool calling.
    Handles all three query categories: direct, multi_step, domain_specific.
    """
    formatted = []
    
    for item in data:
        # Get the tool call string (already formatted)
        tool_call = item.get("tool_call", "")
        if not tool_call and "expected_tools" in item:
            # Build from expected_tools if tool_call not present
            tool_call = make_tool_sequence(item["expected_tools"])
        
        messages = [
            {"role": "system", "content": FINBOT_SYSTEM_PROMPT},
            {"role": "user", "content": item["query"]},
            {"role": "assistant", "content": tool_call},
        ]
        
        # Format using tokenizer's chat template WITH tools
        try:
            text = tokenizer.apply_chat_template(
                messages,
                tools=FINBOT_TOOLS,
                tokenize=False,
                add_generation_prompt=False,
            )
        except Exception as e:
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False,
            )
        
        formatted.append({
            "text": text, 
            "messages": messages,
            "category": item.get("category", "direct"),
            "query": item["query"],
            "tool_call": tool_call,
            "num_tools": item.get("num_tools", 1),
        })
    
    return Dataset.from_list(formatted)


# Prepare dataset from flattened training data
train_dataset = prepare_training_dataset(train_flat, tokenizer)

print(f"✓ Training dataset prepared: {len(train_dataset)} examples")
print(f"\n📊 By category:")
cat_counts = {}
for ex in train_flat:
    cat = ex.get("category", "direct")
    cat_counts[cat] = cat_counts.get(cat, 0) + 1
for cat, count in cat_counts.items():
    print(f"   {cat}: {count}")

print(f"\n📝 Sample training example:")
sample = train_dataset[0]
print(f"   Category: {sample['category']}")
print(f"   Query: {sample['query'][:80]}...")
print(f"   Tool call: {sample['tool_call'][:100]}...")

✓ Training dataset prepared: 114 examples

📊 By category:
   direct: 38
   multi_step: 38
   domain_specific: 38

📝 Sample training example:
   Category: direct
   Query: Find the latest news about Disney....
   Tool call: <tool_call>
{"name": "get_news", "arguments": {"ticker": "DIS"}}
</tool_call>...


## Section 7: Configure LoRA Adapters & Training

In [16]:
# =============================================================================
# CONFIGURE LORA ADAPTERS
# =============================================================================

model = FastLanguageModel.get_peft_model(
    model,
    r=config.LORA_R,
    lora_alpha=config.LORA_ALPHA,
    lora_dropout=config.LORA_DROPOUT,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    use_gradient_checkpointing="unsloth",
)

# Print trainable parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"✓ LoRA configured (r={config.LORA_R}, alpha={config.LORA_ALPHA})")
print(f"  Trainable: {trainable:,} / {total:,} ({trainable/total*100:.2f}%)")

Unsloth 2025.12.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


✓ LoRA configured (r=16, alpha=16)
  Trainable: 29,933,568 / 1,830,055,936 (1.64%)


In [17]:
# =============================================================================
# SFT TRAINING (Native Tool Call Format)
# =============================================================================
from trl import SFTTrainer, SFTConfig

# Dataset already has "text" field formatted with tools via apply_chat_template
# No additional formatting needed

sft_output = OUTPUT_DIR / "sft"
sft_config = SFTConfig(
    output_dir=str(sft_output),
    num_train_epochs=config.SFT_EPOCHS,
    per_device_train_batch_size=config.SFT_BATCH_SIZE,
    gradient_accumulation_steps=config.SFT_GRADIENT_ACCUMULATION,
    learning_rate=config.SFT_LEARNING_RATE,
    warmup_ratio=0.1,
    logging_steps=10,
    save_strategy="epoch",
    fp16=False,  # Model is loaded in bf16
    bf16=True,   # Match model precision
    dataset_text_field="text",  # Use pre-formatted text field
    max_seq_length=config.MAX_SEQ_LENGTH,
    seed=config.RANDOM_SEED,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,  # Already formatted with native tool calls
    args=sft_config,
)

print(f"✓ SFT Trainer configured (native tool call format)")
print(f"  Epochs: {config.SFT_EPOCHS}")
print(f"  Batch size: {config.SFT_BATCH_SIZE}")
print(f"  Gradient accumulation: {config.SFT_GRADIENT_ACCUMULATION}")
print(f"  Effective batch: {config.SFT_BATCH_SIZE * config.SFT_GRADIENT_ACCUMULATION}")
print(f"  Training samples: {len(train_dataset)}")

Unsloth: Tokenizing ["text"] (num_proc=24):   0%|          | 0/114 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.
✓ SFT Trainer configured (native tool call format)
  Epochs: 3
  Batch size: 4
  Gradient accumulation: 4
  Effective batch: 16
  Training samples: 114


In [19]:
# =============================================================================
# RUN TRAINING
# =============================================================================
print("Starting SFT training...")
print(f"Training on {len(train_dataset)} examples for {config.SFT_EPOCHS} epochs")

train_result = trainer.train()

print(f"\n✓ Training complete!")
print(f"  Final loss: {train_result.training_loss:.4f}")

# Save model
model.save_pretrained(sft_output / "final")
tokenizer.save_pretrained(sft_output / "final")
print(f"  Model saved to {sft_output / 'final'}")

The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting SFT training...
Training on 114 examples for 3 epochs


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 114 | Num Epochs = 3 | Total steps = 24
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 29,933,568 of 3,115,872,256 (0.96% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.327800
20,1.181700



✓ Training complete!
  Final loss: 1.2332
  Model saved to /workspace/zgx-onboard/models/checkpoints/finbot-tool-calling/sft/final


## Section 8: Evaluate TUNED Model (After Fine-Tuning)

Same comprehensive logging as base model evaluation for direct comparison.

In [20]:
# =============================================================================
# EVALUATE TUNED MODEL (Three-Tier Evaluation)
# =============================================================================
FastLanguageModel.for_inference(model)

print("Running TUNED model evaluation across three query categories...")
print("Comparing against BASE model performance...")

tuned_results_df, tuned_category_metrics, tuned_summary = run_three_tier_evaluation(
    model, tokenizer, EVAL_QUERIES,
    stage="tuned",
    run_name=f"tuned-{datetime.now().strftime('%Y%m%d_%H%M%S')}"
)

Running TUNED model evaluation across three query categories...
Comparing against BASE model performance...

███████████████████████████████████████████████████████████████████████████████████████████████
█  TUNED MODEL - THREE-TIER EVALUATION
█─────────────────────────────────────────────────────────────────────────────────────────────█
█  Samples: 36 | Categories: direct, multi_step, domain_specific
███████████████████████████████████████████████████████████████████████████████████████████████

═══════════════════════════════════════════════════════════════════════════════════════════════
TEST 1/36: ✅ PASS | 🎯 DIRECT
═══════════════════════════════════════════════════════════════════════════════════════════════

📝 QUERY: "Can you give me a summary of what Amazon does?"

🎯 EXPECTED (1 tool):
   • get_company_profile(ticker=AMZN)

🤖 MODEL OUTPUT (FULL):
   ┌───────────────────────────────────────────────────────────────────────────────────────────┐
   │ <tool_call>                     

## Section 9: Compare BASE vs TUNED Performance

In [21]:
# =============================================================================
# THREE-TIER COMPARISON: BASE vs TUNED
# =============================================================================

print("\n" + "█"*95)
print("█" + " "*35 + "FINAL COMPARISON" + " "*40 + "█")
print("█"*95)

# Overall metrics comparison
print("\n┌" + "─"*93 + "┐")
print(f"│ {'OVERALL METRICS':<40} {'BASE':<25} {'TUNED':<25} │")
print("├" + "─"*93 + "┤")
print(f"│ {'Overall Pass Rate':<40} {base_summary['overall_pass_rate']*100:>22.1f}% {tuned_summary['overall_pass_rate']*100:>22.1f}% │")
print(f"│ {'LLM Judge Score':<40} {base_summary['judge_avg']*100:>22.1f}% {tuned_summary['judge_avg']*100:>22.1f}% │")
print("└" + "─"*93 + "┘")

# Per-category comparison
print("\n┌" + "─"*93 + "┐")
print(f"│ {'BY CATEGORY (Pass Rate)':<40} {'BASE':<25} {'TUNED':<25} │")
print("├" + "─"*93 + "┤")

for cat in ["direct", "multi_step", "domain_specific"]:
    base_m = base_category_metrics.get(cat, {"total": 0, "pass": 0})
    tuned_m = tuned_category_metrics.get(cat, {"total": 0, "pass": 0})
    
    base_rate = (base_m["pass"] / base_m["total"] * 100) if base_m["total"] > 0 else 0
    tuned_rate = (tuned_m["pass"] / tuned_m["total"] * 100) if tuned_m["total"] > 0 else 0
    
    emoji = {"direct": "🎯", "multi_step": "🔗", "domain_specific": "🧠"}.get(cat, "")
    print(f"│ {emoji} {cat:<38} {base_rate:>22.1f}% {tuned_rate:>22.1f}% │")

print("└" + "─"*93 + "┘")

# Improvement summary
print(f"\n📈 IMPROVEMENT SUMMARY:")
overall_imp = (tuned_summary['overall_pass_rate'] - base_summary['overall_pass_rate']) * 100
judge_imp = (tuned_summary['judge_avg'] - base_summary['judge_avg']) * 100
print(f"   Overall Pass Rate: {overall_imp:+.1f}%")
print(f"   LLM Judge Score:   {judge_imp:+.1f}%")

print(f"\n   By Category:")
for cat in ["direct", "multi_step", "domain_specific"]:
    base_m = base_category_metrics.get(cat, {"total": 0, "pass": 0})
    tuned_m = tuned_category_metrics.get(cat, {"total": 0, "pass": 0})
    base_rate = (base_m["pass"] / base_m["total"]) if base_m["total"] > 0 else 0
    tuned_rate = (tuned_m["pass"] / tuned_m["total"]) if tuned_m["total"] > 0 else 0
    imp = (tuned_rate - base_rate) * 100
    emoji = {"direct": "🎯", "multi_step": "🔗", "domain_specific": "🧠"}.get(cat, "")
    print(f"   {emoji} {cat}: {imp:+.1f}%")

# Log comparison to MLflow
with mlflow.start_run(run_name=f"comparison-{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
    mlflow.log_metrics({
        "base_overall_pass": base_summary['overall_pass_rate'],
        "tuned_overall_pass": tuned_summary['overall_pass_rate'],
        "improvement_overall": tuned_summary['overall_pass_rate'] - base_summary['overall_pass_rate'],
        "base_judge": base_summary['judge_avg'],
        "tuned_judge": tuned_summary['judge_avg'],
        "improvement_judge": tuned_summary['judge_avg'] - base_summary['judge_avg'],
    })
    
    # Per-category improvements
    for cat in ["direct", "multi_step", "domain_specific"]:
        base_m = base_category_metrics.get(cat, {"total": 0, "pass": 0})
        tuned_m = tuned_category_metrics.get(cat, {"total": 0, "pass": 0})
        base_rate = (base_m["pass"] / base_m["total"]) if base_m["total"] > 0 else 0
        tuned_rate = (tuned_m["pass"] / tuned_m["total"]) if tuned_m["total"] > 0 else 0
        mlflow.log_metrics({
            f"base_{cat}_pass": base_rate,
            f"tuned_{cat}_pass": tuned_rate,
            f"improvement_{cat}": tuned_rate - base_rate,
        })

print("\n" + "█"*95)


███████████████████████████████████████████████████████████████████████████████████████████████
█                                   FINAL COMPARISON                                        █
███████████████████████████████████████████████████████████████████████████████████████████████

┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│ OVERALL METRICS                          BASE                      TUNED                     │
├─────────────────────────────────────────────────────────────────────────────────────────────┤
│ Overall Pass Rate                                          77.8%                   77.8% │
│ LLM Judge Score                                            89.4%                   88.9% │
└─────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│ BY CATEGORY (Pass Rate)                  B

In [23]:
# =============================================================================
# PER-QUERY COMPARISON (FULL OUTPUT)
# =============================================================================

print("\n" + "="*80)
print("DETAILED PER-QUERY COMPARISON")
print("="*80)

for i in range(len(EVAL_QUERIES)):
    base_row = base_results_df.iloc[i]
    tuned_row = tuned_results_df.iloc[i]
    
    base_ok = base_row["overall_pass"]
    tuned_ok = tuned_row["overall_pass"]
    
    # Status indicator
    if not base_ok and tuned_ok:
        status = "🔧 FIXED"
    elif base_ok and not tuned_ok:
        status = "💔 REGRESSED"
    elif base_ok and tuned_ok:
        status = "✅ BOTH PASS"
    else:
        status = "❌ BOTH FAIL"
    
    print(f"\n{'─'*80}")
    print(f"Query {i+1}: {status}")
    print(f"{'─'*80}")
    print(f"Query:    {base_row['query']}")
    print(f"Expected: {base_row['expected_raw']}")
    
    print(f"\n  BASE MODEL OUTPUT:")
    print(f"  ┌{'─'*74}┐")
    for line in str(base_row['predicted_raw']).split('\n'):
        while len(line) > 72:
            print(f"  │ {line[:72]} │")
            line = line[72:]
        print(f"  │ {line:<72} │")
    print(f"  └{'─'*74}┘")
    print(f"  Match: {'✓' if base_ok else '✗'} | Judge: {base_row['judge_category']} ({base_row['judge_score']})")
    
    print(f"\n  TUNED MODEL OUTPUT:")
    print(f"  ┌{'─'*74}┐")
    for line in str(tuned_row['predicted_raw']).split('\n'):
        while len(line) > 72:
            print(f"  │ {line[:72]} │")
            line = line[72:]
        print(f"  │ {line:<72} │")
    print(f"  └{'─'*74}┘")
    print(f"  Match: {'✓' if tuned_ok else '✗'} | Judge: {tuned_row['judge_category']} ({tuned_row['judge_score']})")

print("\n" + "="*80)


DETAILED PER-QUERY COMPARISON

────────────────────────────────────────────────────────────────────────────────
Query 1: ✅ BOTH PASS
────────────────────────────────────────────────────────────────────────────────
Query:    Can you give me a summary of what Amazon does?
Expected: <tool_call>
{"name": "get_company_profile", "arguments": {"ticker": "AMZN"}}
</tool_call>

  BASE MODEL OUTPUT:
  ┌──────────────────────────────────────────────────────────────────────────┐
  │ <tool_call>                                                              │
  │ {"name": "get_company_profile", "arguments": {"ticker": "AMZN"}}         │
  │ </tool_call>                                                             │
  └──────────────────────────────────────────────────────────────────────────┘
  Match: ✓ | Judge: perfect (1.0)

  TUNED MODEL OUTPUT:
  ┌──────────────────────────────────────────────────────────────────────────┐
  │ <tool_call>                                                            

## Section 10: Test with Live Queries

In [24]:
# =============================================================================
# INTERACTIVE TESTING (Native Tool Calling)
# =============================================================================

test_queries = [
    "What's the current price of Google?",
    "Show me Apple's stock performance over the last quarter",
    "Get analyst recommendations for AMD",
    "Tell me about Netflix's company profile",
    "How has Bitcoin performed this year?",  # Edge case - not a stock
    "Show me Microsoft's 6 month history",
]

print("\n" + "="*90)
print("LIVE QUERY TESTING (TUNED MODEL - Native Tool Calling)")
print("="*90)

for i, query in enumerate(test_queries):
    response, prompt = generate_response(model, tokenizer, query, return_prompt=True)
    parsed = parse_tool_call(response)
    
    print(f"\n{'─'*90}")
    print(f"TEST {i+1}: {query}")
    print(f"{'─'*90}")
    
    print(f"\n📝 USER QUERY: \"{query}\"")
    
    print(f"\n🤖 MODEL RAW OUTPUT (FULL):")
    print(f"   ┌{'─'*86}┐")
    for line in response.split('\n'):
        while len(line) > 84:
            print(f"   │ {line[:84]} │")
            line = line[84:]
        print(f"   │ {line:<84} │")
    print(f"   └{'─'*86}┘")
    
    print(f"\n🔍 PARSED TOOL CALL:")
    print(f"   Parse Success: {'✓' if parsed['parse_success'] else '✗'}")
    print(f"   Parse Method:  {parsed.get('parse_method') or 'N/A'}")
    print(f"   Tool Name:     {parsed['tool_name'] or 'None'}")
    print(f"   Ticker:        {parsed['ticker'] or 'None'}")
    print(f"   Period:        {parsed['period'] or 'None'}")
    print(f"   Arguments:     {json.dumps(parsed.get('arguments', {}))}")
    
    # Show formatted tool call
    if parsed['parse_success']:
        print(f"\n✅ Formatted: {format_tool_call_for_display(parsed)}")
    else:
        print(f"\n❌ Could not parse tool call from response")

print("\n" + "="*90)


LIVE QUERY TESTING (TUNED MODEL - Native Tool Calling)

──────────────────────────────────────────────────────────────────────────────────────────
TEST 1: What's the current price of Google?
──────────────────────────────────────────────────────────────────────────────────────────

📝 USER QUERY: "What's the current price of Google?"

🤖 MODEL RAW OUTPUT (FULL):
   ┌──────────────────────────────────────────────────────────────────────────────────────┐
   │ <tool_call>                                                                          │
   │ {"name": "get_current_price", "arguments": {"ticker": "GOOGL"}}                      │
   │ </tool_call>                                                                         │
   └──────────────────────────────────────────────────────────────────────────────────────┘

🔍 PARSED TOOL CALL:
   Parse Success: ✓
   Parse Method:  native_tool_call
   Tool Name:     get_current_price
   Ticker:        GOOGL
   Period:        None
   Arguments:    

## Summary

This notebook demonstrates **targeted fine-tuning for tool-calling** using a three-tier evaluation methodology, inspired by research showing that small language models can outperform larger models on specific tasks when properly fine-tuned.

### Research Context

Based on ["Small Language Models for Efficient Agentic Tool Calling"](https://arxiv.org/abs/2512.15943), which showed a 350M parameter model achieving **77.55% pass rate** on ToolBench (vs. ChatGPT-CoT at 26%), this experiment explores domain-specific financial tool calling.

### Methodology

**Three-Tier Query Categories:**
| Category | Description | Challenge |
|----------|-------------|-----------|
| 🎯 **Direct** | Single tool, explicit params | Easy - baseline |
| 🔗 **Multi-Step** | Sequential tool chains | Medium - planning |
| 🧠 **Domain-Specific** | Implicit financial reasoning | Hard - domain knowledge |

**Expanded Tool Set (10 tools):**
- Price: `get_current_price`, `get_historical_data`
- Fundamentals: `get_company_profile`, `get_earnings_history`, `get_dividend_history`
- Analysis: `get_analyst_recommendations`, `get_news`
- Ownership: `get_insider_trades`, `get_institutional_holders`
- Comparative: `compare_stocks`

### Evaluation Metrics

1. **Tool Selection Accuracy**: Correct tool(s) selected
2. **Sequence Accuracy**: Correct order for multi-step queries
3. **Parameter Accuracy**: Correct ticker, period, etc.
4. **Overall Pass Rate**: Complete correctness
5. **LLM-as-Judge**: Semantic quality scoring

### Key Features

✅ **Three-Tier Evaluation**: Direct, Multi-Step, Domain-Specific queries  
✅ **10 Expanded Tools**: Increased complexity and ambiguity  
✅ **Multi-Tool Sequences**: Tests planning and ordering  
✅ **Gemini-Generated Data**: High-quality synthetic training examples  
✅ **MLflow Integration**: Per-category metrics in UI  
✅ **Comprehensive Logging**: Full model output, no truncation